 Introduction to CRPropa3 with examples
=====

## Prologue

The field of cosmic rays intersects various physical topics since cosmic rays, when they traverse the universe, are affected by the whole range of phenomena, from particularities of galactic environments to the cosmic microwave background and the expansion of the universe. To investigate and to better understand their behaviour and nature in such a complex context a Monte Carlo type of numerical analysis imposes itself as a logical approach. **CRPropa3** is a numerical framework which fits that purpose. It is a freely available GPLv3-licensed, highly flexible and modular code which preserves the crucial requirement of a Monte Carlo method - the code performance. Because of its modular nature, CRPropa3 can be used to propagate other types of particles like neutrinos or photons, but it is primarily developed for cosmic rays.

This introductory text will try to explain basics concepts, give an overview of available modules, parameters and settings while demonstrating good practices on how to use CRPropa, and eventually try to lure the reader to discover and read the source code of the framework itself. In the same time, the text can lead to a deeper understanding of ultra-high energy cosmic rays. Yet, one should always keep in mind that a computer code can never substitute the richness of the reality, so understanding underlying assumptions and limitations of code is crucial to discover new layers of nature.

Although the most of the code is written in C++, CRPropa can be easily used and extended with Python. The Python interface is also the recommended way how to use the framework, and it will be used in the tutorial. Therefore, intermediate knowledge of Python is somewhat necessary to follow the text.

## Content

* [**Installation and usage inside Jupyter**](Install.ipynb)
    * Virtualenv - a virtual environment for Python
    * Installing or building dependencies
    * Build and install CRPropa
    * Deleting CRPropa
    * Jupyter - an interactive notebook
* [**Basic concepts**](Basics.ipynb)
    * Candidates and modules
    * ModuleList and sources
    * Observer and output methods
* [**Propagation in 3D and magnetic fields**](3Dpropagation.ipynb)
* Further reading

## Further reading

- Tutorials on [CRPropa's wiki](https://github.com/CRPropa/CRPropa3/wiki)